# CvxPy

In [1]:
#!pip install cvxpy

In [2]:
import time
import numpy as np
import cvxpy as cvx

In [3]:
import matplotlib
import matplotlib.pyplot as plt

/home/elias/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
matplotlib.use('Agg')
params = {'legend.fontsize': 20,
          'legend.handlelength': 4,
          "axes.labelsize": 45,
          "xtick.labelsize": 25,
          "ytick.labelsize": 25,
          "lines.linewidth": 4,
           "axes.titlesize":30}
matplotlib.rcParams.update(params)

## 1. CVXPY

CVXPY is a Python-embedded modeling language for convex optimization problems. It allows you to express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

Below there are examples of CVXPY usage.

### 1.1. Unconstrained QP

$$\min_{x \in \mathbb{R}^n} \frac{1}{2}x^\top A x + b^\top x$$

In [5]:
n = 100
np.random.seed(41)
A = np.random.randn(n, n)
A = A.T @ A
b = np.random.randn(n)

In [11]:
# x is variable of diminsion n
x = cvx.Variable(n)
x

Variable((100,), var6)

In [21]:
# Objective function
obj = 1/2 * cvx.quad_form(x, A) + b @ x
obj = cvx.Minimize(obj)

In [27]:
# Problem
problem = cvx.Problem(obj)
problem.solve(verbose=True, solver="OSQP", eps_abs=1e-8, eps_rel=1e-8)

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Oct 21 08:44:14 PM: Your problem has 100 variables, 0 constraints, and 0 parameters.
(CVXPY) Oct 21 08:44:14 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 21 08:44:14 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 21 08:44:14 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 21 08:44:14 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 21 08:44:14 PM: Compiling problem (target solver=OSQP).
(CV

np.float64(-16.453510008799316)

In [28]:
x.value

array([-3.15658716e+00,  3.46431319e+00,  8.38580529e+00,  6.63168349e-01,
        2.17174870e+00, -4.71078344e+00, -1.01199829e+01, -8.07786113e-01,
       -7.19971073e+00, -3.93316697e-01, -6.23999881e+00, -1.75996496e-01,
        1.19733412e+01,  5.55755114e+00,  1.46779142e+01, -5.99166468e+00,
       -6.02009341e+00, -3.30181337e-01, -1.44852742e+01,  1.19158388e+00,
       -3.61174476e+00, -1.22402889e+00,  2.18802191e+00,  5.19542280e+00,
        1.30517346e+00, -7.23760192e+00, -1.12158986e+01, -8.13300361e+00,
       -1.81581037e+00, -1.21527969e+01, -1.34785639e+01,  4.63072052e+00,
       -9.42042009e+00, -5.66938571e-01,  3.44562696e+00, -3.16258945e+00,
       -9.39718528e+00, -1.05508417e+01,  7.79818825e+00, -1.70381263e+00,
        6.14765408e+00,  2.69907357e+00,  3.18833871e+00, -8.24090806e-01,
       -1.95130214e+00, -7.11332120e-01,  1.82889924e+00, -9.76581358e+00,
        5.79155091e+00,  1.31634518e+01,  1.55664729e-02, -1.40214859e+01,
        1.35364328e+01, -

In [29]:
np.linalg.norm(A @ x.value + b)

np.float64(8.273668625615326e-11)

$$\min \|Ax+b\|^2 + \sum\limits_{i=1}^n\sqrt{1+x_i^2}$$

In [ ]:
n = 100
m = 10
A = np.random.randn(m, n)
b = np.random.randn(m)
x = cvx.Variable(n)
obj = cvx.norm(A@x+b)**2 + #...
prob = cvx.Problem(cvx.Minimize(obj))
prob.solve()

### 1.2. QCQP

$$\min_{x \in \mathbb{R}^n} \frac{1}{2}x^\top A x + b^\top x$$

$$\text{s.t. } \frac{1}{2}x^\top A_i x + b_i^\top x+c_i \leq 0, i=\overline{1,m}$$

$$Gx=h$$

where $G\in\mathbb{R}^{d\times n}.$

In [51]:
np.random.seed(1)
n = 100
m = 1
d = 5
A = np.random.randn(n, n)
A = A.T @ A
b = np.random.randn(n)

G = np.random.randn(d, n)
h = np.random.randn(d)

A_list = [np.random.randn(n, n) for _ in range(m)]
A_list = [Ai.T @ Ai for Ai in A_list]
b_list = [np.random.randn(n) for _ in range(m)]
c_list = [np.random.randn() for _ in range(m)]

In [52]:
x = cvx.Variable(n)
obj = 1/2 * cvx.quad_form(x, A) + b @ x
obj = cvx.Minimize(obj)

In [53]:
constrains_lin = [G @ x == h]
constraints_quad = [1/2 * cvx.quad_form(x, Ai) + bi @ x + ci <= 0
                    for Ai, bi, ci in zip(A_list, b_list, c_list)]
constraints = constrains_lin + constraints_quad

In [55]:
# Problem
problem = cvx.Problem(obj, constraints)
result = problem.solve(verbose=True, solver="SCS")

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Oct 21 09:09:23 PM: Your problem has 100 variables, 6 constraints, and 0 parameters.
(CVXPY) Oct 21 09:09:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 21 09:09:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 21 09:09:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 21 09:09:23 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 21 09:09:23 PM: Compiling problem (target solver=SCS).
(CVX

In [56]:
x = x.value


In [57]:
x.shape

(100,)

In [58]:
1/2 * x.T @ A_list[0] @ x + b_list[0].T @ x + c_list[0]

np.float64(2.79253287160941e-09)

In [59]:
np.linalg.norm(G @ x - h)/5

np.float64(9.950900309866763e-15)